## Fleiss Kappa Calculation


<h2>Task: </h2>
<p>This notebook details the manual implementation of Fleiss’s k as described in Part 2 of http://courses.ischool.berkeley.edu/i290-dds/s17/hw/hw1.pdf </p>


In [93]:
import pandas as pd
import numpy as np
#import the data
datafile = 'hw1.part1.2.txt'
data = pd.read_csv(datafile,delimiter='\t',header=None) 
data.head()


,0,1,2
0,item:0,annotator:0,label:0
1,item:0,annotator:1,label:1
2,item:0,annotator:2,label:0
3,item:1,annotator:0,label:0
4,item:1,annotator:1,label:0


In [94]:
#data[0] = data[0].apply(lambda x: int(x[5:]))
#data[1] = data[1].apply(lambda x: int(x[10:]))
#data[2] = data[2].apply(lambda x: int(x[6:]))

#data.head()
#Confirm How many unique items, annotators and labels we have

items  = data[0].unique() #Number of items
annot = data[1].unique() #Number of Annotators
labels = data[2].unique() #Number of Labels

no_items = len(items) #Number of items
no_annot = len(annot) #Number of Annotators
no_labels = len(labels) #Number of Annotators

print('Number of items: ',no_items) 
print('Number of Annotators: ', no_items,' specifically: ',data[1].unique()) 
print('Number of Labels',no_labels,' specifically: ',data[2].unique()) 

Number of items:  100
Number of Annotators:  100  specifically:  ['annotator:0' 'annotator:1' 'annotator:2']
Number of Labels 3  specifically:  ['label:0' 'label:1' 'label:2']


In [95]:
#Create items by labels default matrix. 
#g = np.zeros([100,3], np.int32)
g = np.zeros([len(items),len(labels)], np.int32) #Rows correspond to the number of items. Columns with number of labels
default_matrix = pd.DataFrame(g, index=items,columns=labels) #index corresponds to the item. Column labels with labels
default_matrix.shape

(100, 3)

In [96]:
default_matrix.head()

,label:0,label:1,label:2
item:0,0,0,0
item:1,0,0,0
item:2,0,0,0
item:3,0,0,0
item:4,0,0,0


In [97]:
#Group by Item and Label
data_grouped = data.groupby([0,2])
data_grouped.first()


1
0       2                   
item:0  label:0  annotator:0
        label:1  annotator:1
item:1  label:0  annotator:0
item:10 label:0  annotator:1
        label:1  annotator:0
        label:2  annotator:2
item:11 label:0  annotator:0
item:12 label:0  annotator:0
item:13 label:2  annotator:0
item:14 label:1  annotator:1
        label:2  annotator:0
item:15 label:0  annotator:0
        label:2  annotator:1
item:16 label:0  annotator:1
        label:1  annotator:0
item:17 label:1  annotator:0
        label:2  annotator:1
item:18 label:0  annotator:0
        label:1  annotator:1
item:19 label:0  annotator:0
        label:2  annotator:1
item:2  label:0  annotator:1
        label:1  annotator:0
item:20 label:0  annotator:1
        label:1  annotator:0
item:21 label:1  annotator:1
        label:2  annotator:0
item:22 label:0  annotator:0
item:23 label:2  annotator:0
item:24 label:1  annotator:0
...                      ...
item:84 label:1  annotator:0
item:85 label:0  annotator:0
        label:2  annotator:2
item:86 label:1  annotator:0
        label:2  annotator:1
item:87 label:1  annotator:0
        label:2  annotator:2
item:88 label:1  annotator:0
item:89 label:2  annotator:0
item:9  label:0  annotator:0
        label:2  annotator:1
item:90 label:0  annotator:2
        label:2  annotator:0
item:91 label:2  annotator:0
item:92 label:0  annotator:0
        label:1  annotator:1
item:93 label:0  annotator:0
        label:1  annotator:1
        label:2  annotator:2
item:94 label:1  annotator:0
item:95 label:0  annotator:1
        label:1  annotator:0
item:96 label:0  annotator:1
        label:1  annotator:0
item:97 label:0  annotator:1
        label:2  annotator:0
item:98 label:0  annotator:2
        label:1  annotator:0
item:99 label:0  annotator:0
        label:1  annotator:2

[183 rows x 1 columns]

In [98]:
for item in data_grouped:
    print(item)
    break

(('item:0', 'label:0'),         0            1        2
0  item:0  annotator:0  label:0
2  item:0  annotator:2  label:0)


In [99]:
for item in data_grouped:
    print(item[0])
    print(len(item[1]))
    break

('item:0', 'label:0')
2


In [100]:
# 'Count' value below  will be used to update values of default matrix from 0 to a count of how many times 
#an item was labelled with a specific label

for item in data_grouped:
    row_index = item[0][0] # e.g. 'item:0'
    col_index = item[0][1] # e.g. 'label:0'
    count = len(item[1])   # e.g   2- The item 0 is labelled 0 by 2 annotators
    default_matrix[col_index][row_index] = count

#Print out default_matrix after the updates. NB: Row-wise totals must be equal number of annotators (3 in this case)
#This can be used to confirm the values for a few item-label pairs.
default_matrix.head()

,label:0,label:1,label:2
item:0,2,1,0
item:1,3,0,0
item:2,2,1,0
item:3,1,0,2
item:4,0,3,0


In [101]:
from functools import *

#Calculate Pi
#For each row, calculate the sum of squares for each element.Pi is calculated by subtracting the number of annotators 
#from that sum and dividing by product of the number of annotators and the number of annotators -1
def Pi_cal(row):
    #print(row)
    sum_of_squares = sum(map((lambda u: u **2), [x for x in row if x]))
    Pi= (sum_of_squares - no_annot)/(no_annot*(no_annot-1))
    return Pi

arr=[]
for x in default_matrix.values:
    arr.append(Pi_cal(x))

default_matrix['Pi'] =arr
 
default_matrix.head()

#default_matrix['Pi'] = default_matrix[labels].map(lambda x: Pi_cal(x) )

,label:0,label:1,label:2,Pi
item:0,2,1,0,0.333333
item:1,3,0,0,1.000000
item:2,2,1,0,0.333333
item:3,1,0,2,0.333333
item:4,0,3,0,1.000000


In [102]:
#Add extra row for totals.
final = default_matrix.append(pd.Series(default_matrix.sum(numeric_only=True), name = "totals"))
final.tail()

,label:0,label:1,label:2,Pi
item:96,2.0,1.0,0.0,0.333333
item:97,1.0,0.0,2.0,0.333333
item:98,1.0,2.0,0.0,0.333333
item:99,2.0,1.0,0.0,0.333333
totals,113.0,83.0,104.0,48.000000


In [103]:
#Calculate the mean of the column Pi
Pm = final.iloc[-1]['Pi']/no_items
Pm

0.48000000000000037

In [104]:
#Calculate Pj which is the product of all items by number of annotators. Example 100 items by 3 annotators = 300
Pj = int(no_items*no_annot)

#Pe is the sum of the square of each item of the last row ( i.e.sum of each column) divided by Pj
Pe =sum([((final.iloc[-1][labels[x]])/Pj)**2  for x in range(len(labels)) ])
Pe

0.33860000000000001

In [105]:
#Calculate Fleiss Coppa 
k = (Pm-Pe)/(1-Pe)
k

0.21378893256728207